In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi
from importlib import reload

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

# np.warnings.filterwarnings('error', category=np.VisibleDeprecationWarning)

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_MetOx_33eV_RCP_m30-28_new.root")
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_MetOx_33eV_LCP_m30-28_new.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_MetOx_33eV_RCP_m30-28_new.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_MetOx_33eV_LCP_m30-28_new.root")

DIR = ["if_twoelec/","if_e[0]_valid/"]
EN = ["NO_gate","30%_en_e1","50%_en_e1","80%_en_e1","e12_sym"]
EN_opt = 0
# the next layer is L or R, selected later
ID_PECD=["PECD_e[e1](theta)","PECD_e[e2](theta)","PECD_pesum(theta)","PECD_e[e1](theta)_phi","PECD_e[e2](theta)_phi","PECD_pesum(theta)_phi", "PECD_sym(theta)"]
ID_COS=["cos(theta)_e[e1]","cos(theta)_e[e2]","cos(theta)_pesum","cos(theta)_e[e1]_phi","cos(theta)_e[e2]_phi","cos(theta)_pesum_phi","cos(theta)_sym"]

#twoelec valid
loc_two="angular_distr_el/"+DIR[0]+EN[EN_opt]+"/"
#just e[0]
loc_e0="angular_distr_el/"+DIR[1]+EN[EN_opt]+"/"

save_path="../PYTHON_graphs/OUTPUTS/doubleionization/"

In [ ]:
#commands for uproot single graphs loading
fileRCR[loc_two+ID_PECD[0]].all_members
fileRCR[loc_two+ID_PECD[0]].axis("x").centers() #N
fileRCR[loc_two+ID_PECD[0]].to_numpy()

In [ ]:
#TO SELECT e1, e2, esum
ID_P=5
ID_C=5

PECD_RCR = fileRCR[loc_two+ID_PECD[ID_P]].to_numpy()[0]
cos_RCR = fileRCR[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_RCR_err = fileRCR[loc_two+ID_COS[ID_C]].errors()

PECD_RCL = fileRCL[loc_two+ID_PECD[ID_P]].to_numpy()[0]
cos_RCL = fileRCL[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_RCL_err = fileRCL[loc_two+ID_COS[ID_C]].errors()

PECD_SCR = fileSCR[loc_two+ID_PECD[ID_P]].to_numpy()[0]
cos_SCR = fileSCR[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_SCR_err = fileSCR[loc_two+ID_COS[ID_C]].errors()

PECD_SCL = fileSCL[loc_two+ID_PECD[ID_P]].to_numpy()[0]
cos_SCL = fileSCL[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_SCL_err = fileSCL[loc_two+ID_COS[ID_C]].errors()

ctheta = fileRCR[loc_two+ID_PECD[ID_P]].axis("y").centers()
beta = fileRCR[loc_two+ID_PECD[ID_P]].axis("x").centers()

print("ctheta shape = ", ctheta.shape)
print("beta shape = ", beta.shape)
print("PECD shape = ", PECD_RCR.shape)

PECD_RCR_norm = fugi.normalise_with_err(PECD_RCR)
cos_RCR_norm = fugi.normalise_with_err(cos_RCR)
PECD_RCL_norm = fugi.normalise_with_err(PECD_RCL)
cos_RCL_norm = fugi.normalise_with_err(cos_RCL)
PECD_SCR_norm = fugi.normalise_with_err(PECD_SCR)
cos_SCR_norm = fugi.normalise_with_err(cos_SCR)
PECD_SCL_norm = fugi.normalise_with_err(PECD_SCL)
cos_SCL_norm = fugi.normalise_with_err(cos_SCL)

#cos: SHOULD BE FLIPPED, BUT THE IDEA IS THAT THE HELICITY HAS NOT BEEN REVERSED DURING THE EXPERIMENT
cos_S = cos_SCR+cos_SCL #np.flip(cos_SCL) 
cos_R = cos_RCR+cos_RCL #np.flip(cos_RCL)
cos_S_norm = fugi.normalise_with_err(cos_S)
cos_R_norm = fugi.normalise_with_err(cos_R)

cPECD_tot_R = np.divide(np.subtract(cos_RCR_norm,cos_RCL_norm),np.add(cos_RCR_norm,cos_RCL_norm))
cPECD_tot_R_err = fugi.error_calc(cos_RCR,cos_RCL)

cPECD_tot_S = np.divide(np.subtract(cos_SCR_norm,cos_SCL_norm),np.add(cos_SCR_norm,cos_SCL_norm))
cPECD_tot_S_err = fugi.error_calc(cos_SCR,cos_SCL)

cPECD_tot_RS = np.divide(np.subtract(cos_R_norm,cos_S_norm),np.add(cos_R_norm,cos_S_norm))
cPECD_tot_RS_err = fugi.error_calc(cos_R,cos_S)

#PECD maps: SHOULD BE FLIPPED, BUT THE IDEA IS THAT THE HELICITY HAS NOT BEEN REVERSED DURING THE EXPERIMENT
PECD_S = PECD_SCR+PECD_SCL #np.flip(PECD_SCL,axis=0)
PECD_R = PECD_RCR+PECD_RCL # np.flip(PECD_RCL,axis=0)

PECD_S_norm = fugi.normalise_with_err(PECD_S)
PECD_R_norm = fugi.normalise_with_err(PECD_R)

PECD_tot_RS = np.divide(np.subtract(PECD_R_norm,PECD_S_norm),np.add(PECD_R_norm,PECD_S_norm))
# PECD_tot = np.divide(np.subtract(PECD_S,PECD_R),np.add(PECD_S,PECD_R))

PECD_tot_R = np.divide(np.subtract(PECD_RCR_norm,PECD_RCL_norm),np.add(PECD_RCR_norm,PECD_RCL_norm))
PECD_tot_S = np.divide(np.subtract(PECD_SCR_norm,PECD_SCL_norm),np.add(PECD_SCR_norm,PECD_SCL_norm))

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs,ax=fugi.plot_interpolation(beta,ctheta,PECD_tot_RS.T, xstep=0.01, ystep=0.01, ax=ax, cmap=cmap_temp, gaussian=10.)
# cs=ax.pcolormesh(PECD_tot_RS.T, cmap=cmap_temp)

ax.set_title(" PECD R - S @ 33 eV")
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('cos(\u03B2) [adm]')
ax.set_ylabel('cos(\u03D1) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([cPECD_en.min(),cPECD_en.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('PECD')
# ax.set_aspect(aspect)
# fig.savefig(save_path+"enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECDmap_R.png", dpi=144, transparent=False)


In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs,ax=fugi.plot_interpolation(beta,ctheta,PECD_tot_R.T, xstep=0.01, ystep=0.01, ax=ax, cmap=cmap_temp, gaussian=10.)
# cs=ax.pcolormesh(PECD_tot_R.T, cmap=cmap_temp)

ax.set_title(" PECD S CR-CL @ 33 eV")
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('cos(\u03B2) [adm]')
ax.set_ylabel('cos(\u03D1) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([cPECD_en.min(),cPECD_en.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('PECD')
# ax.set_aspect(aspect)
# fig.savefig(save_path+"enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECDmap_R.png", dpi=144, transparent=False)


In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
cs,ax=fugi.plot_interpolation(beta,ctheta,PECD_tot_S.T, xstep=0.01, ystep=0.01, ax=ax, cmap=cmap_temp, gaussian=10.)
# cs=ax.pcolormesh(PECD_tot_S.T, cmap=cmap_temp)

ax.set_title(" PECD R CR-CL @ 33 eV")
# aspect=(phiM.max()/cosM.max())
ax.set_xlabel('cos(\u03B2) [adm]')
ax.set_ylabel('cos(\u03D1) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([cPECD_en.min(),cPECD_en.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('PECD')
# ax.set_aspect(aspect)
# fig.savefig(save_path+"enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECDmap_R.png", dpi=144, transparent=False)


In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta;y_data = cPECD_tot_RS;y_err = cPECD_tot_RS_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title("R - S @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/PECD_random_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta; y_data = cPECD_tot_R; y_err = cPECD_tot_R_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title("R CR-CL @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/PECD_random_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta; y_data = cPECD_tot_S; y_err = cPECD_tot_S_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title("S CR-CL @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/PECD_random_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
#INTESNITY OF THE COS(THETA)
#SINGLE ELICITIES
import functions_minuit as fumi

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = ctheta;y_data = cos_SCL;y_err = cos_SCL_err

m1=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m1,ax1,custom_xlim)
ax1.set_title("S CL 33 eV")
ax1.set_xlabel('cos(\u03D1) [adm]')
ax1.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m1.parameters, m1.values, m1.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax1.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax1.set_aspect(aspect)

print("CL fitting and covarience \n", m1, "\n")

x_data = ctheta;y_data = cos_SCR;y_err = cos_SCR_err

m2=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m2,ax2,custom_xlim)
ax2.set_title("S CR 33 eV")
ax2.set_xlabel('cos(\u03D1) [adm]')
ax2.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m2.parameters, m2.values, m2.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax2.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax2.set_aspect(aspect)


print("CR fitting  and covarience \n", m2)

# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_single_hel_"+CH+"_"+en+"_exp.png", figsize=(20, 10), dpi=144, transparent=False)

In [ ]:
#INTESNITY OF THE COS(THETA)
#SINGLE ELICITIES
import functions_minuit as fumi

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = ctheta;y_data = cos_RCL;y_err = cos_RCL_err

m1=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m1,ax1,custom_xlim)
ax1.set_title("R CL 33 eV")
ax1.set_xlabel('cos(\u03D1) [adm]')
ax1.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m1.parameters, m1.values, m1.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax1.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax1.set_aspect(aspect)

print("CL fitting and covarience \n", m1, "\n")

x_data = ctheta;y_data = cos_RCR;y_err = cos_RCR_err

m2=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m2,ax2,custom_xlim)
ax2.set_title("R CR 33 eV")
ax2.set_xlabel('cos(\u03D1) [adm]')
ax2.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m2.parameters, m2.values, m2.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax2.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax2.set_aspect(aspect)


print("CR fitting  and covarience \n", m2)

# fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_single_hel_"+CH+"_"+en+"_exp.png", figsize=(20, 10), dpi=144, transparent=False)